In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
import io
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *
import tiktoken


from general_config import *
from ml_tracker import MLTracker
from aso_config import *
from aso_localizer import *
#from marketing_localizer import *
#from marketing_config import *
#from in_game_localizer import InGameLocalizer
#from InGame_Config import * 

In [0]:
%run "./authenticationScript"

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
request =  {"RowFingerprint":"DMM_ASO_TEST_OCTOBER",
                    "Timestamp":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"9/4/25",
                    'LocType':"ASO", 
                    'Game':"Disney Magic Match", 
                    'TargetLanguages':"",
                    "URL":DMM_URL,
                    "QAFlag":""}


cfg = {
    "input": 
        {
            "required_tabs": ["ios","android"],
            "ios_header_rows": 3, 
            "android_header_rows": 3
        },
    ##add more formatting data for header rows
    "char_limit_policy": "strict",
    "output_sheets":
        ["formatted ios", "formatted android", "long results",'wide results'],
    ##add more formatting info for output sheets
    }

In [0]:
#mlflow.active_run()
#mlflow.end_run()

In [0]:
localizer = ASOLocalizer(request = request, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
localizer.game_description

In [0]:
localizer.validate_inputs() # GOOD
inputs = localizer.load_inputs()
preprocessed = localizer.preprocess() #Good
prompts = localizer.build_prompts(preprocessed) #Good


In [0]:
raw_results = localizer.translate(prompts) #Good


In [0]:
postprocessed = localizer.postprocess(raw_results)
formatted, artifacts = localizer._format_results(postprocessed ) #Good
#localizer.write_outputs(formatted) #Good

In [0]:
#localizer._write_wide_results(formatted['wide'])

In [0]:
localizer._write_long_results(formatted['long'])

In [0]:
#formatted['long']

In [0]:
#formatted['wide']

In [0]:
localizer._write_wide_results(formatted['wide'])

In [0]:
#####

In [0]:
long_tab = localizer.sh.worksheet('long results')
long_tab.get_all_values()

#localizer.unioned_long

In [0]:
long_ready_to_write = localizer.unioned_long[["RowFingerprint","row_idx",'row_id','en_char_limit','game','platform','type_desc','en_US','language','language_cd','target_char_limit','translation']].values.tolist()


long_range = f"A2:L{len(long_ready_to_write)+1}"

In [0]:
long_tab.batch_update([{'range':long_range,'values':long_ready_to_write}])

In [0]:
wide_order = ['row_id','en_char_limit','game','platform','type_desc','en_US',*localizer.lang_cds]

ordered_wide = localizer.unioned_wide[wide_order]


In [0]:
len(localizer.unioned_wide)

In [0]:
data_wide_range = f"A2:P{len(ordered_wide)+1}"
wide_data = ordered_wide.values.tolist()

wksht = localizer.sh.worksheet("wide results")
try:
    wksht.batch_update([{'range':data_wide_range, 'values':wide_data}])
    print("Writing wide results...DONE!")
except Exception as e:
    print(e)
    print("Couldn't write wide results to tab")

In [0]:
localizer._write_wide_results(localizer.unioned_wide)

In [0]:
#localizer._write_wide_results()

In [0]:
#run = mlflow.active_run()
#mlflow.end_run()

In [0]:
#mlflow.active_run()

In [0]:
#Tryring run()

#localizer.run()

In [0]:
#Stepwise 

In [0]:
localizer_step = ASOLocalizer(request = request, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
localizer_step.request['URL']

In [0]:
localizer.validate_inputs() # GOOD
inputs = localizer.load_inputs()
preprocessed = localizer.preprocess() #Good

In [0]:
prompts = localizer.build_prompts(preprocessed) #Good
raw_results = localizer.translate(prompts) #Good

In [0]:
postprocessed = localizer.postprocess(raw_results)
formatted, artifacts = localizer._format_results(postprocessed) #Good
localizer.write_outputs(formatted) #Good

In [0]:
#preprocessed['French']

In [0]:
list(preprocessed.keys())

In [0]:
### only join on en_US, not row_idx

In [0]:
prompts['French']

In [0]:

raw_results = localizer_step.translate(prompts) #Good

In [0]:
#raw_results

In [0]:
#localizer.prepped

In [0]:
# parse_results 
# 
# 
# post processs

In [0]:

"""
results_dict = {}

for key, item in (dict(zip(localizer.languages, raw_results))).items():
    print(key)
    #print(item)
    ## Parsed json results 
    results = localizer._parse_model_json_block(item)

    # convert to df
    df = pd.DataFrame(results)

    # add to key
    results_dict[key] = df"""

In [0]:
#ins = dict(zip(localizer.languages, localizer.prepped_holder))

In [0]:
#ins['French']

In [0]:
#localizer.prepped_holder

In [0]:
#localizer.prepped['']

In [0]:
#results_dict['French']

In [0]:
#ins['French']

In [0]:
#for key, value in results_dict.items():
#    inputs = ins[key]
    # parse keys or add "en_US" to outputs
    #df = df.rename(columns={localizer.lang_map[key]:'translation'},inplace=False)
    #df = inputs.merge(df,on=['row_idx'],how='left')



In [0]:
#joined

In [0]:
"""
joined_dict = {}
joined_holder = []
for key, df in results_dict.items():
    inputs = ins[key]
    df = df.rename(columns={localizer.lang_map[key]:'translation'},inplace=False)
    #TODO: if "en_US" added to outputs, join on ['row_idx','en_US']
    joined = inputs.merge(df,on=['row_idx'],how='left')
    joined_dict[key] = joined
    joined_holder.append(joined)


long_results = pd.concat(joined_holder)"""

In [0]:
"""pd.concat(joined_holder)"""

In [0]:
#joined_dict['French']

In [0]:
#pd.DataFrame(results)

In [0]:
processed = localizer_step.postprocess(raw_results)
#print(localizer_step.write_outputs(processed))
#localizer.format_re#sults(processed)


#unioned_long, unioned_wide = localizer.format_results(processed)

In [0]:
processed

In [0]:
processed

In [0]:
wide_results, long_results = localizer_step.format_results(processed)

In [0]:
wide_results

In [0]:
dbutils.notebook.exit()

In [0]:
wide_order = ['row_id','en_char_limit','game','platform','type_desc','en_US',*localizer.lang_cds]

long_order = ["RowFingerprint","row_idx",'row_id','en_char_limit','game','platform','type_desc','en_US','language','language_cd','target_char_limit','translation']


ordered_wide = wide_results[wide_order]
ordered_long = long_results[long_order]


data_long_range = f"A2:L{len(ordered_long)+1}"
long_data = ordered_long.values.tolist()


data_wide_range = f"A2:P{len(ordered_wide)+1}"
wide_data = ordered_wide.values.tolist()

wide_wksht = localizer.sh.worksheet('wide results')
long_wksht = localizer.sh.worksheet('long results')


wide_wksht.batch_update([{'range':data_wide_range, 'values':wide_data}])
long_wksht.batch_update([{'range':data_long_range, 'values':long_data}])